# **Draw LTP**

In [40]:
import ezdxf
import pandas as pd
from ezdxf import zoom

file_name='NV-19A.MD-MOD'
df=pd.read_csv(f'{file_name}.CSV', usecols=[0,1,2,3,4], names=["est", "x", "y", "z", "des"], header=None)
df['xy']=df.apply(lambda j: (j['x'],j['y']),axis=1)
df['xyz']=df.apply(lambda j: (j['x'],j['y'],j['z']),axis=1)
df.head()

,est,x,y,z,des,xy,xyz
0,R17,366227.7474,8.710953e+06,3508.3157,PP,"(366227.7474, 8710952.944299998)","(366227.7474, 8710952.944299998, 3508.3157)"
1,1,366232.5783,8.710981e+06,3508.6108,L,"(366232.5783, 8710980.8992)","(366232.5783, 8710980.8992, 3508.6108)"
2,2,366232.5687,8.710979e+06,3508.6680,L,"(366232.5687, 8710979.0821)","(366232.5687, 8710979.0821, 3508.668)"
3,3,366232.4063,8.710977e+06,3508.6655,L,"(366232.4063, 8710977.0599)","(366232.4063, 8710977.0599, 3508.6655)"
4,4,366232.4180,8.710976e+06,3508.6551,L,"(366232.418, 8710976.3112)","(366232.418, 8710976.3112, 3508.6551)"


In [41]:
cases = ['L','L1','L2','L3','T','P']
filt = df['des'].isin(cases)
df_ltp = df.loc[filt,['des','xyz']]
df_ltp.reset_index(drop=True, inplace=True)
df_ltp

,des,xyz
0,L,"(366232.5783, 8710980.8992, 3508.6108)"
1,L,"(366232.5687, 8710979.0821, 3508.668)"
2,L,"(366232.4063, 8710977.0599, 3508.6655)"
3,L,"(366232.418, 8710976.3112, 3508.6551)"
4,L,"(366231.809, 8710972.674699998, 3508.6026)"
...,...,...
71,L,"(366231.1747, 8710962.402699998, 3488.3921)"
72,L,"(366231.2973, 8710962.193299998, 3488.2157)"
73,L,"(366230.8326, 8710958.9441, 3488.2474)"
74,L,"(366231.0098, 8710957.5388, 3488.1657)"


In [42]:
df_ltp['des'][1]

'L'

In [43]:
#Group dataframe by 'des'
grouped=df_ltp.groupby('des')
grouped.groups.keys()

dict_keys(['L', 'L2', 'L3', 'T'])

In [44]:
doc=ezdxf.new(dxfversion='R2000', setup=True)
msp=doc.modelspace()
doc.layers.new(name='L', dxfattribs={'color':1})
doc.layers.new(name='L1', dxfattribs={'color':2})
doc.layers.new(name='L2', dxfattribs={'color':5})
doc.layers.new(name='L3', dxfattribs={'color':6})
doc.layers.new(name='T', dxfattribs={'color':181})
doc.layers.new(name='P', dxfattribs={'color':7})

<class 'ezdxf.entities.layer.Layer'> LAYER(#8E)

In [45]:
#find start : end of sections

ltp_list = df_ltp['des'].tolist()

idx=[]
jo = 7777
vo = 'any'

for i,v in enumerate(ltp_list):
    if v!=vo:
        idx=idx+[i]
        jo=i
        vo=v

print(idx)

from pprint import pprint

for i in range(len(idx)):
    if i < len(idx)-1:
        points = df_ltp['xyz'][idx[i]:idx[i+1]].tolist()
        msp.add_polyline3d(points=points, close=False, dxfattribs={'layer':df_ltp['des'][idx[i]]})
        #pprint(points)
    else:
        points = df_ltp['xyz'][idx[i]:].tolist()
        #pprint(points)
        msp.add_polyline3d(points=points, close=False, dxfattribs={'layer':df_ltp['des'][idx[i]]})

zoom.extents(msp)
doc.saveas('ltp.dxf')

[0, 16, 30, 45, 60]


In [46]:
#0,4,7,9
#test_list=[1,2,3,4,1,2,3,1,2,1,2,3,4,5,6,7]
#test_list=[1,2,3,4]

test_list = ['L','L','L','P','P','L1','L1','L1','L1','T','L','L','L3']
#test_list = ['L','L','L']

In [47]:
idx=[]
jo = 7777
vo = 'any'

for i,v in enumerate(test_list):
    if v!=vo:
        idx=idx+[i]
        jo=i
        vo=v
idx        

[0, 3, 5, 9, 10, 12]

In [48]:
for i in range(len(idx)):
    if i < len(idx)-1:
        print(test_list[idx[i]:idx[i+1]])
    else:
        print(test_list[idx[i]:])


['L', 'L', 'L']
['P', 'P']
['L1', 'L1', 'L1', 'L1']
['T']
['L', 'L']
['L3']
